In [ ]:
import pandas as pd
import os
from tqdm import tqdm
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
import random
import tensorflow as tf
from utils import *

In [ ]:
train_cv = pd.read_csv('data/train.csv')
train_cv = train_cv.drop('ID', axis =1 )

In [ ]:
def Build_Model(boosting_type, learning_rate, n_estimator):
    model = LGBMRegressor(boosting_type = boosting_type, learning_rate = learning_rate, n_estimators=n_estimator, verbose = -1)
    
    return model

In [ ]:
kfold = KFold(n_splits=5)

results = {'Feature_n' : [], 'boosting_type' : [], 'learning_rate': [], 'n_estimator': [], 'boost_from_average': [], 'RMSE': []} 

In [ ]:
for feature_n in tqdm([30, 50, 70, 100, 130, 150, 170, 200, 230, 250, 274]):

    for train_idx, test_idx in kfold.split(train_cv):
        
        train, test = train_cv.iloc[train_idx], train_cv.iloc[test_idx] 

        train, test = One_hot_encoder(train, test,variable=['Gender', 'Education_Status', 'Employment_Status',
                                                            'Industry_Status', 'Occupation_Status', 'Race',
                                                            'Hispanic_Origin', 'Martial_Status', 'Household_Status',
                                                            'Household_Summary', 'Citizenship', 'Birth_Country',
                                                            'Birth_Country (Father)', 'Birth_Country (Mother)', 'Tax_Status', 'Income_Status'])
        
               
        continuous_col = find_continuous_col(train)

        selected_feautre = Feature_selection(train, feature_n=feature_n)

        X_train, y_train = train[selected_feautre], train['Income']
        X_test, y_test = test[selected_feautre], test['Income']
        
        for boosting_type in ['gbdt', 'dart']:
            for learning_rate in [0.01, 0.1, 0.15, 0.3]:    
                for n_estimator in [100, 300, 500, 700]:          
                        
                    model = Build_Model(boosting_type, learning_rate, n_estimator)                        
                    model.fit(X_train, y_train)
                    y_pred = model.predict(X_test)

                    rmse = np.sqrt(mean_squared_error(y_test, y_pred))          
                    
                    results['Feature_n'].append(feature_n)
                    results['boosting_type'].append(boosting_type)
                    results['learning_rate'].append(learning_rate)
                    results['n_estimator'].append(n_estimator)
                    
                    results['RMSE'].append(rmse)

model_results = pd.DataFrame(results)
model_results

In [ ]:
model_results.sort_values('RMSE')